In [1]:
from seisLM.model import pretrained_models
from seisLM.model import multidim_wav2vec2
import seisbench.data as sbd
import torch
from seisLM.model.multidim_wav2vec2 import MultiDimWav2Vec2ForPreTraining
from seisLM.utils import project_path

from transformers import Wav2Vec2Config

In [2]:
model_config_path = project_path.gitdir(
) + '/seisLM/configs/pretrain/pretrained_model_config.json'
model_config = Wav2Vec2Config.from_pretrained(model_config_path)

model = MultiDimWav2Vec2ForPreTraining(model_config)

In [8]:
from torchinfo import summary

summary(model.wav2vec2.encoder)

Layer (type:depth-idx)                                  Param #
Wav2Vec2EncoderStableLayerNorm                          --
├─Wav2Vec2PositionalConvEmbedding: 1-1                  --
│    └─ParametrizedConv1d: 2-1                          240
│    │    └─ModuleDict: 3-1                             460,928
│    └─Wav2Vec2SamePadLayer: 2-2                        --
│    └─GELUActivation: 2-3                              --
├─LayerNorm: 1-2                                        480
├─Dropout: 1-3                                          --
├─ModuleList: 1-4                                       --
│    └─Wav2Vec2EncoderLayerStableLayerNorm: 2-4         --
│    │    └─Wav2Vec2SdpaAttention: 3-2                  231,360
│    │    └─Dropout: 3-3                                --
│    │    └─LayerNorm: 3-4                              480
│    │    └─Wav2Vec2FeedForward: 3-5                    123,376
│    │    └─LayerNorm: 3-6                              480
│    └─Wav2Vec2EncoderLayerStabl

In [7]:
pretrained_model.wav2vec2.encoder

Wav2Vec2EncoderStableLayerNorm(
  (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
    (conv): ParametrizedConv1d(
      240, 240, kernel_size=(128,), stride=(1,), padding=(64,), groups=16
      (parametrizations): ModuleDict(
        (weight): ParametrizationList(
          (0): _WeightNorm()
        )
      )
    )
    (padding): Wav2Vec2SamePadLayer()
    (activation): GELUActivation()
  )
  (layer_norm): LayerNorm((240,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-5): 6 x Wav2Vec2EncoderLayerStableLayerNorm(
      (attention): Wav2Vec2SdpaAttention(
        (k_proj): Linear(in_features=240, out_features=240, bias=True)
        (v_proj): Linear(in_features=240, out_features=240, bias=True)
        (q_proj): Linear(in_features=240, out_features=240, bias=True)
        (out_proj): Linear(in_features=240, out_features=240, bias=True)
      )
      (dropout): Dropout(p=0.0, inplace=False)
      (layer_norm): LayerNorm(

In [4]:
config = pretrained_model.config
config.num_labels = 3
classification_model = multidim_wav2vec2.MultiDimWav2Vec2ForFrameClassification(
  config)

# Copy weights from pretrained model (if needed)
classification_model.wav2vec2.load_state_dict(pretrained_model.wav2vec2.state_dict())

classification_model.cuda();


In [5]:
outputs = classification_model(
  input_values=input_values,
  # interp_output_length
  # labels=one_hot_tensor.cuda() #torch.argmax(one_hot_tensor, 1).cuda()
)

In [6]:
outputs.shape

torch.Size([1, 3, 6000])

In [7]:
# labels = torch.argmax(outputs[0], 2)
# torch.argmax(labels, 1)

# outputs.shape
# config.getattr('interp_output_length', None)

In [3]:
from transformers import Wav2Vec2Config

model_config1 = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base")
model_config2 = Wav2Vec2Config.from_pretrained("patrickvonplaten/wav2vec2-base-v2")

/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [7]:
# model_config2.save_pretrained('config.json')

Wav2Vec2Config.from_pretrained('/home/liu0003/Desktop/projects/seisLM/configs/pretrain/pretrained_model_config.json')

Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    256,
    256
  ],
  "conv_kernel": [
    3,
    3
  ],
  "conv_stride": [
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 307

In [9]:
from seisLM.utils.project_path import gitdir

In [15]:
# bs = model_config.get('batch_size', None)
from ml_collections import config_dict

training_config = config_dict.ConfigDict()

# training_config.batch_size = 32
training_config.get('batch_size', None)
# print(bs)

In [17]:
import seisbench.data as sbd

dir(sbd)

['BenchmarkDataset',
 'Bucketer',
 'ChunkedDummyDataset',
 'DummyDataset',
 'ETHZ',
 'GEOFON',
 'GeometricBucketer',
 'ISC_EHB_DepthPhases',
 'InstanceCounts',
 'InstanceCountsCombined',
 'InstanceGM',
 'InstanceNoise',
 'Iquique',
 'LFEStacksCascadiaBostock2015',
 'LFEStacksMexicoFrank2014',
 'LFEStacksSanAndreasShelly2017',
 'LenDB',
 'MLAAPDE',
 'Meier2019JGR',
 'MultiWaveformDataset',
 'NEIC',
 'OBS',
 'OBST2024',
 'PNW',
 'PNWAccelerometers',
 'PNWExotic',
 'PNWNoise',
 'Ross2018GPD',
 'Ross2018JGRFM',
 'Ross2018JGRPick',
 'SCEDC',
 'STEAD',
 'TXED',
 'WaveformDataWriter',
 'WaveformDataset',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'base',
 'dummy',
 'ethz',
 'geofon',
 'instance',
 'iquique',
 'isc_ehb',
 'lendb',
 'lfe_stacks',
 'neic',
 'obs',
 'obst2024',
 'pnw',
 'scedc',
 'stead',
 'txed']